In [72]:
# !pip install torch
import json
import os
import glob 
import sys
import torch
import requests
from transformers import * 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit

EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.001

device = torch.device('cpu')
### Bert
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased').to(device)
def get_bert_embeddings(word):
    word = ' '.join(word.replace('_',' ').replace('#','').split())
    input_ids = torch.tensor([tokenizer.encode(word, add_special_tokens=True)])                                                                                                                       
    input_ids = input_ids.to(device)                                                                                
    import pdb                                                                                                       
    #pdb.set_trace()                                                                                               
    with torch.no_grad():                                                                                        
        import pdb                                                                                                 
        #pdb.set_trace()                                                                                             
        last_hidden_states = bert_model(input_ids)[0].cpu().detach().numpy()  # Models outputs are now tuples       
                                                                                                                    
    output= np.reshape(last_hidden_states, (-1, last_hidden_states.shape[-1])).sum(0)
    return output


In [67]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(y_pred)

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

class model(nn.Module):
    
    def __init__(self, input_dim):
        super(model, self).__init__()
        
        self.input_dim = input_dim
        self.relu = nn.ReLU()
        self.ln1 = nn.Linear(self.input_dim, 2*self.input_dim)
        self.ln2 = nn.Linear(2*self.input_dim, 2*self.input_dim)
        self.ln3 = nn.Linear(2*self.input_dim, 2*self.input_dim)
        self.ln4 = nn.Linear(2*self.input_dim, self.input_dim)
        self.ln5 = nn.Linear(self.input_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, feature):
        feature = self.ln1(feature)
        feature = self.relu(feature)
        feature = self.ln2(feature)
        feature = self.relu(feature)
        feature = self.ln3(feature)
        feature = self.relu(feature)
        feature = self.ln4(feature)
        feature = self.relu(feature)
        feature = self.ln5(feature)
        return self.sigmoid(feature)
    
    def loss(self, pred, rel):
        return ((rel - pred)**2).sum()

## train data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)


<ipython-input-67-3d0bc8939500>:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


model(
  (relu): ReLU()
  (ln1): Linear(in_features=768, out_features=1536, bias=True)
  (ln2): Linear(in_features=1536, out_features=1536, bias=True)
  (ln3): Linear(in_features=1536, out_features=1536, bias=True)
  (ln4): Linear(in_features=1536, out_features=768, bias=True)
  (ln5): Linear(in_features=768, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [29]:
from tqdm.notebook import tqdm
x = json.load(open('abtract_data/output_0_50000.json'))
# Load more data to x in forder abtract_data

X = []
Y = []
for i in tqdm(x):
    X.append(get_bert_embeddings(i['abtract']))
    Y.append(i['isTechnology'])
X = np.asarray(X)
Y = np.asarray(Y).astype(int)
from sklearn.model_selection import StratifiedShuffleSplit

cv = StratifiedShuffleSplit(1,test_size=0.3, random_state=42)
for train_idx, test_idx in cv.split(X, Y):
    X_train=X[train_idx]
    Y_train=Y[train_idx]
    X_test=X[test_idx]
    Y_test=Y[test_idx]
train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(Y_train))
test_data = testData(torch.FloatTensor(X_test))
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [77]:
# model = model(768)
model.apply(weights_init)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = model.loss(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

<ipython-input-67-3d0bc8939500>:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


Epoch 1: | Loss: 7.18569 | Acc: 88.455
Epoch 2: | Loss: 1.54545 | Acc: 97.364
Epoch 3: | Loss: 1.54545 | Acc: 97.364
Epoch 4: | Loss: 1.54545 | Acc: 97.364
Epoch 5: | Loss: 1.54545 | Acc: 97.455
Epoch 6: | Loss: 1.54545 | Acc: 97.545
Epoch 7: | Loss: 1.54545 | Acc: 97.364
Epoch 8: | Loss: 1.54545 | Acc: 97.455
Epoch 9: | Loss: 1.54545 | Acc: 97.455
Epoch 10: | Loss: 1.54545 | Acc: 97.545
Epoch 11: | Loss: 1.54545 | Acc: 97.364
Epoch 12: | Loss: 1.54545 | Acc: 97.455
Epoch 13: | Loss: 1.54545 | Acc: 97.455
Epoch 14: | Loss: 1.54545 | Acc: 97.273
Epoch 15: | Loss: 1.54545 | Acc: 97.364
Epoch 16: | Loss: 1.54545 | Acc: 97.455
Epoch 17: | Loss: 1.54545 | Acc: 97.364
Epoch 18: | Loss: 1.54545 | Acc: 97.364
Epoch 19: | Loss: 1.54545 | Acc: 97.455
Epoch 20: | Loss: 1.54545 | Acc: 97.364
Epoch 21: | Loss: 1.54545 | Acc: 97.455


KeyboardInterrupt: 

In [59]:
for X_batch in test_loader:
    y_pred = model(X_batch)
    acc = binary_acc(y_pred, torch.Tensor(Y_test))
print(acc)

tensor(98.)
